In [6]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("kerala.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus0_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()


Navigating to page 2
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


NameError: name 'driver_' is not defined

In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("APSRTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus1_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [3]:
#RSRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("RSRTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus2_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()


Navigating to page 2
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [4]:
#CTU
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("CTU.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus3_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()


Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [5]:
#HRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("HRTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus4_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [2]:
#ASTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("ASSAM.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus5_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [3]:
#SBSTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("SBSTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus6_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [4]:
#WBTC(CTC)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("WBTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus7_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [10]:
#PEPSU
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("PEPSU.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus8_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [11]:
#TSRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load


# Function to retrieve bus route links and route names
def link_route(path):
    LINKS = []
    ROUTE = []
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            paths = driver.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS.append(d)
            for route in paths:
                ROUTE.append(route.text)
            
            # Handle pagination
            try:
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click()
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break

    return LINKS, ROUTE


# Retrieve route links and names
LINKS, ROUTE = link_route("//a[@class='route']")

# Save route data
df_routes = pd.DataFrame({"Route_name": ROUTE, "Route_link": LINKS})
df_routes.to_csv("TSRTC.csv", index=False)
print("Route details saved successfully.")

# Close the first driver
driver.quit()


# Initialize second WebDriver for bus details
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

Bus_names = []
Bus_types = []
Departure = []
Arrival = []
Ratings = []
Total_Duration = []
Prices = []
Seats_Available = []
Route_names = []
Route_links = []

# Loop through route links to extract bus details
for i, r in df_routes.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]
    driver.get(link)
    time.sleep(2)

    try:
        view_buses_button = driver.find_element(By.XPATH, "//div[@class='button']")
        view_buses_button.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(5)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        rating = []

    # Append data to lists
    for bus in bus_name:
        Bus_names.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration.append(total_duration_elem.text)
    for ratings_elem in rating:
        Ratings.append(ratings_elem.text if ratings_elem else "N/A")
    for price_elem in price:
        Prices.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available.append(seats_elem.text)

print("Bus details extracted successfully.")

# Save bus data
data = {
    'Route_name': Route_names,
    'Route_link': Route_links,
    'Bus_name': Bus_names,
    'Bus_type': Bus_types,
    'Departing_time': Departure,
    'Total_duration': Total_Duration,
    'Reaching_time': Arrival,
    'Star_Rating': Ratings,
    'Price': Prices,
    'Seats_Available': Seats_Available
    
    
}
df_buses = pd.DataFrame(data)
df_buses.to_csv("redbus9_details.csv", index=False)
print("Bus details saved successfully.")

# Close the WebDriver
driver.quit()

Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found
Route details saved successfully.
Bus details extracted successfully.
Bus details saved successfully.


In [52]:
import pandas as pd

# Load the existing CSV file
df_buses = pd.read_csv("redbus1_details.csv")

# Add 'State' column with 'Kerala' as the value for all rows
df_buses['State'] = 'Kerala'

# Save the updated DataFrame back to the CSV file
df_buses.to_csv("redbus0_details.csv", index=False)
print("State column added with 'Kerala' in all rows successfully.")


State column added with 'Kerala' in all rows successfully.


In [1]:
from mysql import connector
connection =connector.connect(
    host="localhost",
    user="root",
    password="Mona@999"
)
mycursor = connection.cursor()

In [2]:
query = "Create database if not exists REDBUS_DETAILS"
mycursor.execute(query)

In [3]:
query ="use RED_BUS"
mycursor.execute(query)

In [4]:
import pandas as pd
import mysql.connector

# Reading the CSV files
dfbus1 = pd.read_csv("redbus0_details.csv")
dfbus2 = pd.read_csv("redbus1_details.csv")
dfbus3 = pd.read_csv("redbus2_details.csv")
dfbus4 = pd.read_csv("redbus3_details.csv")
dfbus5 = pd.read_csv("redbus4_details.csv")
dfbus6 = pd.read_csv("redbus5_details.csv")
dfbus7 = pd.read_csv("redbus6_details.csv")
dfbus8 = pd.read_csv("redbus7_details.csv")
dfbus9 = pd.read_csv("redbus8_details.csv")
dfbus10 = pd.read_csv("redbus9_details.csv")

# Concatenating all dataframes into one
dfbus = pd.concat([dfbus1, dfbus2, dfbus3, dfbus4, dfbus5, dfbus6, dfbus7, dfbus8, dfbus9, dfbus10], ignore_index=True)




In [5]:
dfbus.head(10)

,Route_name,Route_link,Bus_name,Bus_type,Departing_time,Total_duration,Reaching_time,Star_Rating,Price,Seats_Available,State
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 581,Super Express Non AC Seater Air Bus (2+2),10:00,08h 40m,18:40,4.2\n97,469.0,1 Seat available,KERALA
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,07h 39m,21:25,2.5\n35,729.0,14 Seats available,KERALA
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 50m,23:05,4.3\n96,627.0,23 Seats available,KERALA
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,10h 00m,01:00,3.6\n17,620.0,18 Seats available,KERALA
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:00,08h 45m,00:45,3.5\n21,567.0,15 Seats available,KERALA
5,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 380,Swift Deluxe Non AC Air Bus (2+2),16:15,08h 45m,01:00,4.6\n7,567.0,19 Seats available,KERALA
6,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 553,Super Express Non AC Seater Air Bus (2+2),16:45,09h 30m,02:15,3.5\n14,640.0,26 Seats available,KERALA
7,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:00,09h 30m,04:30,3.9\n44,640.0,30 Seats available,KERALA
8,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 789,Super Fast Non AC Seater (2+3),20:00,07h 45m,03:45,3.8\n83,473.0,21 Seats available,KERALA
9,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1585,Swift Deluxe Non AC Air Bus (2+2),20:40,09h 30m,06:10,3.3\n62,640.0,27 Seats available,KERALA


In [6]:
dfbus.isna().sum()

Route_name         0
Route_link         0
Bus_name           0
Bus_type           0
Departing_time     0
Total_duration     0
Reaching_time      0
Star_Rating        0
Price              0
Seats_Available    0
State              0
dtype: int64

In [7]:
dfbus.shape

(3310, 11)

In [8]:
dfbus['Route_link'].unique

<bound method Series.unique of 0       https://www.redbus.in/bus-tickets/bangalore-to...
1       https://www.redbus.in/bus-tickets/bangalore-to...
2       https://www.redbus.in/bus-tickets/bangalore-to...
3       https://www.redbus.in/bus-tickets/bangalore-to...
4       https://www.redbus.in/bus-tickets/bangalore-to...
                              ...                        
3305    https://www.redbus.in/bus-tickets/hyderabad-to...
3306    https://www.redbus.in/bus-tickets/hyderabad-to...
3307    https://www.redbus.in/bus-tickets/hyderabad-to...
3308    https://www.redbus.in/bus-tickets/hyderabad-to...
3309    https://www.redbus.in/bus-tickets/hyderabad-to...
Name: Route_link, Length: 3310, dtype: object>

In [9]:
dfbus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Route_name       3310 non-null   object 
 1   Route_link       3310 non-null   object 
 2   Bus_name         3310 non-null   object 
 3   Bus_type         3310 non-null   object 
 4   Departing_time   3310 non-null   object 
 5   Total_duration   3310 non-null   object 
 6   Reaching_time    3310 non-null   object 
 7   Star_Rating      3310 non-null   object 
 8   Price            3310 non-null   float64
 9   Seats_Available  3310 non-null   object 
 10  State            3310 non-null   object 
dtypes: float64(1), object(10)
memory usage: 284.6+ KB


In [10]:
dfbus['Departing_time']=pd.to_datetime(dfbus['Departing_time'],format="%H:%M").dt.strftime("%H:%M")

In [11]:
dfbus['Reaching_time']=pd.to_datetime(dfbus['Reaching_time'],format="%H:%M").dt.strftime('%H:%M')

In [12]:
print(type(dfbus['Departing_time']))

<class 'pandas.core.series.Series'>


In [13]:
print(type(dfbus['Reaching_time']))

<class 'pandas.core.series.Series'>


In [14]:
dfbus['Departing_time']

0       10:00
1       13:46
2       14:15
3       15:00
4       16:00
        ...  
3305    19:00
3306    19:00
3307    19:05
3308    19:45
3309    20:40
Name: Departing_time, Length: 3310, dtype: object

In [15]:
dfbus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Route_name       3310 non-null   object 
 1   Route_link       3310 non-null   object 
 2   Bus_name         3310 non-null   object 
 3   Bus_type         3310 non-null   object 
 4   Departing_time   3310 non-null   object 
 5   Total_duration   3310 non-null   object 
 6   Reaching_time    3310 non-null   object 
 7   Star_Rating      3310 non-null   object 
 8   Price            3310 non-null   float64
 9   Seats_Available  3310 non-null   object 
 10  State            3310 non-null   object 
dtypes: float64(1), object(10)
memory usage: 284.6+ KB


In [16]:
#convert rating to float
dfbus['Star_Rating']

0       4.2\n97
1       2.5\n35
2       4.3\n96
3       3.6\n17
4       3.5\n21
         ...   
3305        5.0
3306        4.5
3307        3.4
3308        3.9
3309        3.6
Name: Star_Rating, Length: 3310, dtype: object

In [17]:
dfbus['Star_Rating'] = dfbus['Star_Rating'].str.split('\n').str[0]
dfbus['Star_Rating'] = pd.to_numeric(dfbus['Star_Rating'], errors='coerce')

In [18]:
dfbus['Star_Rating']

0       4.2
1       2.5
2       4.3
3       3.6
4       3.5
       ... 
3305    5.0
3306    4.5
3307    3.4
3308    3.9
3309    3.6
Name: Star_Rating, Length: 3310, dtype: float64

In [19]:
dfbus.shape

(3310, 11)

In [20]:
dfbus['Star_Rating'].dtype

dtype('float64')

In [21]:
#convert column of seats availability in integer
dfbus['Seats_Available'].dtype

dtype('O')

In [22]:
dfbus['Seats_Available']

0         1 Seat available
1       14 Seats available
2       23 Seats available
3       18 Seats available
4       15 Seats available
               ...        
3305    27 Seats available
3306    25 Seats available
3307    28 Seats available
3308    23 Seats available
3309    18 Seats available
Name: Seats_Available, Length: 3310, dtype: object

In [23]:
dfbus['Seats_Available']=dfbus['Seats_Available'].str.replace("Seats available","").str[0]

In [24]:
dfbus[dfbus['Seats_Available']==" "]

,Route_name,Route_link,Bus_name,Bus_type,Departing_time,Total_duration,Reaching_time,Star_Rating,Price,Seats_Available,State


In [25]:
len(dfbus[dfbus['Seats_Available']==0])

0

In [26]:
dfbus['Seats_Available']

0       1
1       1
2       2
3       1
4       1
       ..
3305    2
3306    2
3307    2
3308    2
3309    1
Name: Seats_Available, Length: 3310, dtype: object

In [27]:
dfbus['Seats_Available']=dfbus['Seats_Available'].astype(int)

In [28]:
#price to decimal
dfbus['Price'].dtype

dtype('float64')

In [29]:
dfbus[dfbus['Price']==" "]

,Route_name,Route_link,Bus_name,Bus_type,Departing_time,Total_duration,Reaching_time,Star_Rating,Price,Seats_Available,State


In [30]:
dfbus['Price'].unique()

array([ 469.  ,  729.  ,  627.  ,  620.  ,  567.  ,  640.  ,  473.  ,
        750.  , 1099.  ,  799.  ,  833.  ,  800.  ,  383.  ,  268.  ,
        269.  ,  230.  ,  261.  ,  220.  ,  293.  ,  209.  ,  699.  ,
        549.  ,  700.  ,  432.  ,  513.  ,  688.  ,  999.  ,  900.  ,
        309.  ,  278.  ,  855.  ,  400.  ,  306.  ,  391.  ,  285.  ,
        273.  ,  494.  ,  398.  ,  356.  ,  419.  ,  319.  ,  601.  ,
        694.  ,  749.  ,  649.  ,  350.  ,  452.  ,  599.  ,  253.  ,
        216.  ,  181.  ,  166.  ,  174.  ,  742.  ,  723.  ,  515.  ,
        477.  ,  502.  ,  555.  ,  648.  ,  650.  ,  332.  ,  475.  ,
        381.  , 1100.  ,  480.  ,  500.  , 1000.  ,  377.  ,  576.  ,
        524.  ,  548.  ,  568.  ,  579.  , 1049.  ,  899.  ,  849.  ,
        348.  ,  426.  ,  586.  ,  600.  ,  386.  ,  341.  ,  360.  ,
        516.  ,  935.  ,  904.  , 1018.  ,  352.  ,  247.  ,  233.  ,
        239.  ,  281.  ,  340.  ,  520.  , 5000.  ,  277.  ,  186.  ,
        229.  ,  256

In [31]:
dfbus['Price']=dfbus['Price'].astype(float).round(2)

In [32]:
dfbus['Price']

0       469.0
1       729.0
2       627.0
3       620.0
4       567.0
        ...  
3305    680.0
3306    580.0
3307    580.0
3308    499.0
3309    699.0
Name: Price, Length: 3310, dtype: float64

In [33]:
dfbus[dfbus['Total_duration']==" "]

,Route_name,Route_link,Bus_name,Bus_type,Departing_time,Total_duration,Reaching_time,Star_Rating,Price,Seats_Available,State


In [34]:
dfbus['Total_duration'].unique()

array(['08h 40m', '07h 39m', '08h 50m', '10h 00m', '08h 45m', '09h 30m',
       '07h 45m', '09h 25m', '10h 40m', '10h 20m', '09h 00m', '08h 25m',
       '08h 15m', '05h 19m', '04h 20m', '04h 55m', '05h 44m', '05h 50m',
       '05h 46m', '05h 30m', '04h 51m', '06h 25m', '05h 35m', '05h 11m',
       '05h 45m', '05h 21m', '05h 25m', '05h 49m', '05h 00m', '06h 10m',
       '06h 15m', '05h 40m', '03h 50m', '06h 00m', '04h 30m', '04h 45m',
       '08h 20m', '08h 35m', '08h 30m', '09h 15m', '09h 40m', '09h 10m',
       '09h 45m', '05h 20m', '03h 41m', '05h 06m', '06h 11m', '06h 01m',
       '05h 31m', '05h 51m', '05h 01m', '05h 15m', '04h 05m', '03h 05m',
       '08h 10m', '07h 50m', '06h 20m', '06h 08m', '06h 33m', '05h 59m',
       '05h 05m', '07h 25m', '06h 55m', '07h 20m', '07h 00m', '06h 05m',
       '06h 29m', '06h 40m', '08h 00m', '07h 15m', '02h 25m', '03h 15m',
       '02h 55m', '03h 20m', '03h 40m', '03h 25m', '03h 45m', '03h 35m',
       '03h 30m', '04h 00m', '02h 35m', '03h 00m', 

In [41]:
dfbus['Bus_type'].unique()

array(['Super Express Non AC Seater Air Bus (2+2)', 'AC MULTI AXLE',
       'SWIFT-GARUDA A/C SEATER BUS', 'Swift Deluxe Non AC Air Bus (2+2)',
       'Super Fast Non AC Seater (2+3)', 'A/C Seater / Sleeper (2+1)',
       'A/C Sleeper (2+1)', 'Non A/C Seater / Sleeper (2+1)',
       'VE A/C Sleeper (2+1)', 'Low Floor AC Seater 2+2',
       'A/C Semi Sleeper (2+2)', 'A/C Seater/Sleeper (2+1)',
       'Bharat Benz A/C Semi Sleeper (2+2)', 'NON A/C Semi Sleeper (2+2)',
       'Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2)',
       'NON A/C Sleeper (2+1)', '2+1 Air Suspension A/C Seater / Sleeper',
       'Bharat Benz A/C Seater /Sleeper (2+1)',
       'A/C Seater Push Back (2+2)', 'Volvo AC Seater (2+2)',
       'Volvo A/C Semi Sleeper (2+2)', 'VE A/C Seater / Sleeper (2+1)',
       'NON A/C Semi Sleeper / Sleeper (2+1)',
       'NON A/C Seater/ Sleeper (2+1)',
       'Super Deluxe Non AC Seater Air Bus (2+2)',
       'Bharat Benz A/C Sleeper (2+1)',
       'SUPER LUXURY (NON-AC, 2 + 2

In [42]:
dfbus[dfbus['Route_name']==" "]

,Route_name,Route_link,Bus_name,Bus_type,Departing_time,Total_duration,Reaching_time,Star_Rating,Price,Seats_Available,State


In [44]:
import numpy as np
dfbus=dfbus.replace({np.nan:None})

In [45]:
dfbus.to_csv("tenstates_details.csv",index=False)

In [46]:
dfbus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Route_name       3310 non-null   object 
 1   Route_link       3310 non-null   object 
 2   Bus_name         3310 non-null   object 
 3   Bus_type         3310 non-null   object 
 4   Departing_time   3310 non-null   object 
 5   Total_duration   3310 non-null   object 
 6   Reaching_time    3310 non-null   object 
 7   Star_Rating      3284 non-null   object 
 8   Price            3310 non-null   float64
 9   Seats_Available  3310 non-null   int64  
 10  State            3310 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 284.6+ KB


SQL

In [36]:
from mysql import connector
connection =connector.connect(
    host="localhost",
    user="root",
    password="Mona@999"
)
mycursor = connection.cursor()

In [37]:
query="create database if not exists redbusproject"
mycursor.execute(query)

In [38]:
query ="use redbusproject"
mycursor.execute(query)

In [39]:
query ="drop table bus_details"
mycursor.execute(query)

In [40]:
mycursor.execute("""
CREATE TABLE IF NOT EXISTS bus_details (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Route_name VARCHAR(255) NULL,
    Route_link VARCHAR(255) NULL,
    Bus_name VARCHAR(255) NOT NULL,
    Bus_type VARCHAR(255) NOT NULL,
    state VARCHAR(255) NOT NULL,
    Departing_time VARCHAR(255) NOT NULL,
    Total_duration VARCHAR(255) NOT NULL,
    Reaching_time VARCHAR(255) NOT NULL,
    Star_Rating FLOAT NULL,
    Price DECIMAL(10,2),
    Seats_available VARCHAR(255) NOT NULL
    
    
    
)
""")

print("Table created successfully")


Table created successfully


In [41]:
query="describe bus_details "
mycursor.execute(query)
for db in mycursor:
    print(db)

('ID', 'int', 'NO', 'PRI', None, 'auto_increment')
('Route_name', 'varchar(255)', 'YES', '', None, '')
('Route_link', 'varchar(255)', 'YES', '', None, '')
('Bus_name', 'varchar(255)', 'NO', '', None, '')
('Bus_type', 'varchar(255)', 'NO', '', None, '')
('state', 'varchar(255)', 'NO', '', None, '')
('Departing_time', 'varchar(255)', 'NO', '', None, '')
('Total_duration', 'varchar(255)', 'NO', '', None, '')
('Reaching_time', 'varchar(255)', 'NO', '', None, '')
('Star_Rating', 'float', 'YES', '', None, '')
('Price', 'decimal(10,2)', 'YES', '', None, '')
('Seats_available', 'varchar(255)', 'NO', '', None, '')


In [47]:
insert_query = """
INSERT INTO bus_details (
    Route_name,
    Route_link,
    Bus_name,
    Bus_type,
    state,
    Departing_time,
    Total_duration,
    Reaching_time,
    Star_Rating,
    Price,
    Seats_available
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
"""

for index, row in dfbus.iterrows():
    mycursor.execute(insert_query, (
        row['Route_name'],
        row['Route_link'],
        row['Bus_name'],
        row['Bus_type'],
        row['State'],
        row['Departing_time'],
        row['Total_duration'],
        row['Reaching_time'],
        row['Star_Rating'],
        row['Price'],
        row['Seats_Available']
    ))

connection.commit()
print("Data inserted successfully!")


Data inserted successfully!
